In [5]:
# 1) 패키지 참조
# chromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# chrome을 제어하기 위한 객체(webdriver)
from selenium import webdriver
# chrome이 웹페이지 로딩을 완료할 때까지 최대 n초간 대기
from selenium.webdriver.support.ui import WebDriverWait
# 원하는 부분을 추출할 수 있는 방식을 지정하는 객체
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from pandas import DataFrame
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time
import datetime as dt

In [ ]:
# 2) 번역할 파일 가져오기
source = None 
#-> 왜 쓰는 걸까? with구문이 없어도 source가 존재할 수 있으니까. 
with open('news.txt','r', encoding='utf-8') as f:
    source = f.readlines()

source

In [7]:
# 3) 웹 브라우저 준비
# -1. 크롬드라이버 자동 설치
chromedriver_autoinstaller.install()
# -2. driver = chrome의 객체로서 사용
driver = webdriver.Chrome()
# -3. 크롬브라우저가 준비될 때까지 최대 5초씩 대기
driver.implicitly_wait(5)
# -4. 파파고 페이지로 이동
driver.get("https://papago.naver.com/")
time.sleep(3)

In [8]:
# 4) 번역 수행하기
# -1. 번역 결과를 담을 파일 준비 (현재날짜를 파일명으로 함 = datetime모듈)
fileName = dt.datetime.now().strftime("%y%m%d_%H%M%S.md") #마크다운
# -2. 파일 쓰기모드로 불러오기 -> 행 별로 반복문 실행
with open(fileName, "w", encoding='utf-8') as f:
    for s in source:
        s = s.strip()
        if not s: # 번역할 내용이 없으면 반복문 계속 실행
            continue
        # print(s)

        # <번역할 원문 입력하는 요소> 취득
        # CSS_SELECTOR에 의해 #txtSource 값을 추출한 element값이 없을때까지 반복문 실행 => driver(chrome)은 반복문 실행동안 3초 대기
        txtSource = WebDriverWait(driver, 3).until(lambda x : x.find_element(By.CSS_SELECTOR, "#txtSource"))
        #기존 입력내용을 삭제
        txtSource.clear()
        #번역할 한 행을 입력요소에 키보드로 입력(각각의 행을 웹브라우저에 send)
        txtSource.send_keys(s)

        # <번역하기 버튼요소> 취득
        # CSS_SELECTOR에 의해 #btnTranslate 값을 추출한 element값이 없을때까지 반복문 실행 => driver(chrome)은 반복문 실행동안 3초 대기
        btnTranslate = WebDriverWait(driver, 3).until(lambda x : x.find_element(By.CSS_SELECTOR, "#btnTranslate"))
        #번역버튼 클릭
        btnTranslate.click()
        #번역이 되는 동안 1초대기
        time.sleep(1)

        # -------------------------
        # <번역결과가 표시될 요소> 취득
        # CSS_SELECTOR에 의해 #txtTarget 값을 추출한 element값이 없을때까지 반복문 실행 => driver(chrome)은 반복문 실행동안 3초 대기
        txtTarget = WebDriverWait(driver, 3).until(lambda x : x.find_element(By.CSS_SELECTOR, "#txtTarget"))
        # 번역결과 가져오기
        CurrentTrans = txtTarget.text.strip()
        # print(CurrentTrans)
        # 번역이 되는 동안 1초대기
        time.sleep(1)

        # while True: ->왜? 
            #<번역 결과가 표시될 요소> 취득
            # txtTarget = WebDriverWait(driver, 3).until(lambda x : x.find_element(By.CSS_SELECTOR, "#txtTarget"))
            #번역결과 가져오기
            # CurrentTrans = txtTarget.text.strip()
            #가져온 내용이 있고, 이번 번역 결과가 이전 번역 결과와 다르다면 반복 중단
            # if CurrentTrans and CurrentTrans != prevTrans:
            #     prevTrans = CurrentTrans
            #     break
        # -----------------------
        # 번역결과를 파일에 기록
        f.write(s)
        f.write("\n\n")
        f.write("> ")
        f.write(CurrentTrans)
        f.write("\n\n")
        
print("------작업이 종료되었습니다.--------")

------작업이 종료되었습니다.--------
